### 构造可以生成 b state 的量子电路

注意，本文件运行是需要包含的 qiskit 的环境，即 requirements_noise

In [1]:
# 导入必要的库
# from jax import numpy as jnp
# import jax    # 机器学习库
# import optax  # 优化器
import matplotlib.pyplot as plt

import pennylane as qml

from pennylane import numpy as np
import VQLS_until

from qasm_until import export_to_openqasm

In [2]:
b_vec = np.array([1000, 0, -600]).astype(float)

# 相对应的 b 和 x  也要进行维度拓展，以适应 H_matrixs
b_vec_2 = np.zeros((8))
b_vec_2[:3] = b_vec
b_norm = np.linalg.norm(b_vec_2)

# 归一化向量，转回行向量，方便后续计算
b_normalized = (b_vec_2 / b_norm)

print(b_normalized)

rho = b_normalized.reshape(-1,1) @ b_normalized.reshape(1,-1)

print(np.trace(rho))
print(np.trace(rho @ rho))

[ 0.85749293  0.         -0.51449576  0.          0.          0.
  0.          0.        ]
1.0
1.0000000000000002


In [3]:
dev = qml.device("lightning.qubit", wires=3)

# 生成 b_vec_2 的量子电路 ， |b_vec_2 > = U_b |0>
@qml.qnode(dev)
def U_b_circuit(state):
    qml.MottonenStatePreparation(state_vector=state, wires=[0,1,2])
    return qml.state()

# 现有电路图
print(qml.draw(VQLS_until.U_b_circuit, expansion_strategy="device")(b_normalized))


0: ───────────╭●───────────╭●────╭●────╭●──RZ(-0.79)─╭●───────────╭●──────────────╭●─────────────
1: ──RY(0.54)─╰X──RY(0.54)─╰X─╭●─│──╭●─│───RZ(0.79)──╰X──RZ(0.79)─╰X─╭●───────────│────────────╭●
2: ───────────────────────────╰X─╰X─╰X─╰X──RZ(-0.79)─────────────────╰X──RZ(0.79)─╰X──RZ(0.79)─╰X

─────────────╭●─┤  State
─────────────│──┤  State
───RZ(-0.79)─╰X─┤  State


In [4]:
# 定义一个变分电路
@qml.qnode(dev)
def b_circuit():
    qml.RY(0.54, wires=1)
    qml.CNOT([0,1])
    qml.RY(0.54, wires=1)
    qml.CNOT([0,1])
    
    qml.CNOT([1,2])
    qml.CNOT([0,2])
    qml.CNOT([1,2])
    qml.CNOT([0,2])
    
    qml.RZ(-0.79, wires=0) 
    qml.RZ(0.79, wires=1) 
    qml.RZ(-0.79, wires=2) 
    
    qml.CNOT([0,1])
    qml.RZ(0.79, wires=1) 
    qml.CNOT([0,1])
    
    qml.CNOT([1,2])
    qml.RZ(0.79, wires=2)
    qml.CNOT([0,2]) 
    qml.RZ(0.79, wires=2)
    qml.CNOT([1,2]) 
    qml.RZ(-0.79, wires=1) 
    qml.CNOT([0,2]) 

    return qml.state()


In [5]:
b_state = b_circuit()
print(b_state)
print(abs(b_state)**2)

[ 0.79166217-0.33005332j  0.        +0.j         -0.20220286+0.47270479j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]
[0.73566418 0.         0.26433582 0.         0.         0.
 0.         0.        ]


In [6]:
b_normalized**2

tensor([0.73529412, 0.        , 0.26470588, 0.        , 0.        ,
        0.        , 0.        , 0.        ], requires_grad=True)

将电路存储成 openqasm 形式

In [7]:
# 定义电路函数
circuit_fn = b_circuit
# 创建运行节点
qnode = qml.QNode(circuit_fn, dev)

qnode()

# 获取量子电路的量子脚本
quantum_script = qnode.tape

# 导出电路为 OpenQASM 格式
qasm_str = export_to_openqasm(quantum_script)

# 保存到文件
with open("Ub_circuit.qasm", "w") as file:
    file.write(qasm_str)
    
# 导入上述保存的 OpenQASM 格式，并绘图，查看是否一致
my_circuit = qml.from_qasm_file("Ub_circuit.qasm")

drawer = qml.draw(my_circuit)
print(drawer())

1: ──RY(0.54)─╭X──RY(0.54)─╭X─╭●────╭●──RZ(0.79)────────────╭X──RZ(0.79)─╭X─╭●─────────────
0: ───────────╰●───────────╰●─│──╭●─│──╭●─────────RZ(-0.79)─╰●───────────╰●─│────────────╭●
2: ───────────────────────────╰X─╰X─╰X─╰X─────────RZ(-0.79)─────────────────╰X──RZ(0.79)─╰X

────────────╭●──RZ(-0.79)─┤  
────────────│──╭●─────────┤  
───RZ(0.79)─╰X─╰X─────────┤  
